### Github上で Google が管理しているリポジトリをスクレイピングし、以下の情報を取得する

・リポジトリ名

・主要な言語

・スターの数

In [7]:
import sqlite3
import time
import requests
from bs4 import BeautifulSoup

In [8]:
# --- データベース関連の基本設定 ---
# DBファイルの保存先パス
path = ''
# DBファイル名
db_name = 'kadai.db'

# --- 1. データベースとテーブルの準備 ---
print('データベースの準備を開始します...')
try:
    # DB接続オブジェクトの作成
    # データベースファイルが存在しない場合は新規作成される
    conn = sqlite3.connect(path + db_name)
    # SQLを実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()
    
    # SQL文の作成
    # repositoriesテーブルがなければ作成する (IF NOT EXISTS)
    #   - id: 主キー、自動で番号が振られる
    #   - name: リポジトリ名 (TEXT型、NULLを許可しない)
    #   - language: 主要言語 (TEXT型)
    #   - stars: スター数 (INTEGER型、NULLを許可しない)
    sql = '''
    CREATE TABLE IF NOT EXISTS repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        language TEXT,
        stars INTEGER NOT NULL
    );
    '''
    
    # SQL文の実行
    cur.execute(sql)

except sqlite3.Error as e:
    print(f'エラーが発生しました: {e}')

finally:
    # DBへの接続を閉じる
    # tryブロックでエラーが発生しても、必ず接続は閉じられる
    if 'conn' in locals() and conn:
        conn.close()
    print('データベースの準備が完了しました。')

データベースの準備を開始します...
データベースの準備が完了しました。


In [9]:
# --- GitHubからデータをスクレイピング ---
# スクレイピング対象のURL
target_url = 'https://github.com/2y2y-ui/dsprog2'
# 取得したデータを保存するためのリスト
repo_data_list = []

print('スクレイピングを開始します...')
try:
    # サーバーに負荷をかけないよう1秒待機する
    time.sleep(1)
    
    # Webページを取得する
    response = requests.get(target_url)
    # ステータスコードが200番台でない場合に例外を発生させる
    response.raise_for_status()

    # 取得したHTMLをBeautifulSoupで解析する
    soup = BeautifulSoup(response.text, 'html.parser')

    # リポジトリ一覧の要素を取得する （構造に基づいて設定）
    repo_elements = soup.select('div#user-repositories-list ul li')

    # 各リポジトリ要素から情報を抽出するループ
    for repo_elem in repo_elements:
        # リポジトリ名を取得
        name_elem = repo_elem.select_one('h3 > a')
        if not name_elem:
            continue # 名前の要素がなければスキップ
        name = name_elem.text.strip()
        
        # 主要言語を取得
        lang_elem = repo_elem.select_one('span[itemprop="programmingLanguage"]')
        language = lang_elem.text.strip() if lang_elem else 'N/A' # 言語がない場合もある

        # スター数を取得
        star_elem = repo_elem.select_one('a[href*="/stargazers"]')
        if star_elem:
            stars_text = star_elem.text.strip().replace(',', '')
            # 'k'を1000倍して数値に変換
            if 'k' in stars_text:
                stars = int(float(stars_text.replace('k', '')) * 1000)
            else:
                stars = int(stars_text)
        else:
            stars = 0 # スターの要素がない場合は0とする
        
        # 取得したデータをタプルにまとめてリストに追加
        repo_data_list.append((name, language, stars))
        print(f"取得成功: {name}")

except requests.exceptions.RequestException as e:
    print(f'リクエスト中にエラーが発生しました: {e}')

finally:
    print(f'スクレイピングが完了しました。{len(repo_data_list)}件のデータを取得しました。')

スクレイピングを開始します...
スクレイピングが完了しました。0件のデータを取得しました。
